##SVM para classificação:

Entrada: um vetor unificado com as características das duas imagens sendo a última coluna 1 (mesma pessoa) ou 0 (pessoa diferente)

Passos: verificar vetor e definir se 1 ou 0, baseado na lógica da sequência de features

Saída: 1 ou 0

In [1]:
import pandas as pd
import numpy as np

In [157]:
#carregar os dados

hog = pd.read_parquet('./feature_matrix_hog.parquet', engine='pyarrow') 

In [158]:
hog.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,match
0,2.306814,3.954538,3.624993,3.460221,2.636359,3.130676,2.636359,2.306814,1.647724,2.636359,...,0.295641,0.000000,0.295641,0.000000,0.000000,0.147820,0.000000,0.000000,0.147820,True
1,2.193303,2.976625,3.759947,3.603283,4.073276,4.073276,1.253316,1.409980,2.349967,0.939987,...,0.500357,0.333572,0.333572,0.000000,0.333572,0.000000,0.500357,0.000000,0.166786,True
2,3.875630,3.720604,3.100504,3.410554,3.100504,4.030655,1.860302,1.860302,1.550252,1.395227,...,0.482976,0.160992,0.321984,0.160992,0.321984,0.160992,0.160992,0.160992,0.160992,True
3,4.624305,3.545300,3.545300,2.466296,3.391157,2.928726,1.849722,2.158009,1.387291,1.695578,...,0.132290,0.132290,0.132290,0.000000,0.000000,0.000000,0.132290,0.000000,0.132290,True
4,2.934841,4.255520,4.549004,3.962035,3.521809,1.614163,1.614163,1.467421,0.733710,1.173936,...,0.501895,0.167298,0.167298,0.334597,0.167298,0.167298,0.000000,0.000000,0.334597,True


In [159]:
hog = hog.sample(n=100)

In [160]:
hog.shape

(100, 61)

In [161]:
#transformar rotulos em -1 e 1
X = hog.replace(True,1)
X = X.replace(False,-1)

In [162]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,match
1076,1.776087,3.374566,3.196957,4.085001,3.019348,3.374566,2.308914,3.374566,1.776087,2.131305,...,0.000000,0.280036,0.140018,0.000000,0.000000,0.140018,0.000000,0.280036,0.140018,1
765,5.157460,3.352349,2.578730,2.707667,2.965540,2.578730,1.418302,0.902556,1.805111,1.805111,...,0.269165,0.000000,0.000000,0.000000,0.269165,0.000000,0.000000,0.000000,0.134583,1
2596,1.977107,3.163371,2.965661,3.756504,3.558793,3.558793,3.558793,3.163371,2.570239,2.570239,...,0.138022,0.000000,0.138022,0.138022,0.000000,0.000000,0.138022,0.000000,0.138022,1
40,2.194619,3.135171,3.291929,4.702756,3.135171,3.291929,3.448688,2.351378,1.254068,1.254068,...,0.160289,0.641155,0.000000,0.000000,0.000000,0.160289,0.160289,0.000000,0.160289,1
2313,3.168686,5.809258,3.696801,2.288496,3.872839,1.232267,2.992648,1.584343,1.760381,2.464534,...,0.426275,0.426275,0.000000,0.142092,0.000000,0.142092,0.284183,0.000000,0.142092,1


In [163]:
Y = X.iloc[:,-1]

In [164]:
Y.head()

1076    1
765     1
2596    1
40      1
2313    1
Name: match, dtype: int64

In [165]:
#retirar ultima coluna do X
X = X.drop(['match'], axis=1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
1076,1.776087,3.374566,3.196957,4.085001,3.019348,3.374566,2.308914,3.374566,1.776087,2.131305,...,0.420054,0.000000,0.280036,0.140018,0.000000,0.000000,0.140018,0.000000,0.280036,0.140018
765,5.157460,3.352349,2.578730,2.707667,2.965540,2.578730,1.418302,0.902556,1.805111,1.805111,...,0.134583,0.269165,0.000000,0.000000,0.000000,0.269165,0.000000,0.000000,0.000000,0.134583
2596,1.977107,3.163371,2.965661,3.756504,3.558793,3.558793,3.558793,3.163371,2.570239,2.570239,...,0.414065,0.138022,0.000000,0.138022,0.138022,0.000000,0.000000,0.138022,0.000000,0.138022
40,2.194619,3.135171,3.291929,4.702756,3.135171,3.291929,3.448688,2.351378,1.254068,1.254068,...,0.160289,0.160289,0.641155,0.000000,0.000000,0.000000,0.160289,0.160289,0.000000,0.160289
2313,3.168686,5.809258,3.696801,2.288496,3.872839,1.232267,2.992648,1.584343,1.760381,2.464534,...,0.568366,0.426275,0.426275,0.000000,0.142092,0.000000,0.142092,0.284183,0.000000,0.142092


In [166]:
N, ne = X.shape

In [167]:
X

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
1076,1.776087,3.374566,3.196957,4.085001,3.019348,3.374566,2.308914,3.374566,1.776087,2.131305,...,0.420054,0.000000,0.280036,0.140018,0.000000,0.000000,0.140018,0.000000,0.280036,0.140018
765,5.157460,3.352349,2.578730,2.707667,2.965540,2.578730,1.418302,0.902556,1.805111,1.805111,...,0.134583,0.269165,0.000000,0.000000,0.000000,0.269165,0.000000,0.000000,0.000000,0.134583
2596,1.977107,3.163371,2.965661,3.756504,3.558793,3.558793,3.558793,3.163371,2.570239,2.570239,...,0.414065,0.138022,0.000000,0.138022,0.138022,0.000000,0.000000,0.138022,0.000000,0.138022
40,2.194619,3.135171,3.291929,4.702756,3.135171,3.291929,3.448688,2.351378,1.254068,1.254068,...,0.160289,0.160289,0.641155,0.000000,0.000000,0.000000,0.160289,0.160289,0.000000,0.160289
2313,3.168686,5.809258,3.696801,2.288496,3.872839,1.232267,2.992648,1.584343,1.760381,2.464534,...,0.568366,0.426275,0.426275,0.000000,0.142092,0.000000,0.142092,0.284183,0.000000,0.142092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,2.018118,3.302375,3.852771,3.669306,2.201584,4.219702,3.485841,2.568514,2.018118,1.284257,...,0.154925,0.000000,0.309850,0.309850,0.309850,0.000000,0.000000,0.154925,0.154925,0.309850
24,3.571120,2.597178,3.084149,4.058090,2.434854,2.921825,1.785560,1.947883,3.408796,1.785560,...,0.278824,0.139412,0.278824,0.000000,0.418236,0.139412,0.000000,0.000000,0.000000,0.418236
660,3.124491,2.812042,3.124491,3.593164,2.968266,3.593164,1.406021,2.187144,2.968266,3.124491,...,0.276060,0.552119,0.000000,0.138030,0.138030,0.000000,0.138030,0.000000,0.000000,0.138030
1474,3.387113,4.200020,2.032268,2.845175,3.251629,3.116144,1.761299,2.438721,1.490330,2.303237,...,0.140367,0.140367,0.140367,0.000000,0.140367,0.140367,0.000000,0.140367,0.000000,0.280735


In [168]:
Y = np.array(Y)

In [169]:
Y.shape

(100,)

In [170]:
X.shape

(100, 60)

In [171]:
#matriz para kernel linear:
K = np.array(X@X.T)

#matriz para kernel polinomial:
# d = 6 #ordem do polinomio
# K = []
# H = []
# for i in range(N):
#     print('i', i)
#     for j in range(N):
#         print('j', j)
#         K[i][j] = (np.matmul(X[i:],X[j:].T)+1)**d
#         print(K[1][1])
#         H[i][j] = np.matmul(np.matmul(K[i][j], Y[i,:1]), Y[j,:1])

In [172]:
C = 1000
c = -np.ones((1,N))

liminf = np.zeros((N,1))
limsup = C*np.ones((N,1))

In [173]:
Aeq = Y.T
beq = 0
A = []
b = []

In [18]:
!pip install cvxopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:01:01


In [16]:
!pip install quadprog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.4/92.4 kB 1.9 MB/s eta 0:00:00m eta 0:00:01


In [36]:
import cvxopt
import quadprog

In [126]:
#biblioteca quadprog do python: não consegui usar porque não tem a restrição de limite inferior e superior
#P = K ; q = c ; G = A ; h = b ; A = Aeq ; b = beq
def quadprog_solve_qp(P, q, G, h, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = -G.T
        qp_b = -h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

In [174]:
quadprog_solve_qp(K, c, A, b, Aeq, beq)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 100 and the array at index 1 has size 0

In [37]:
#definindo função quadprog equivalente ao quadprog do MatLab (H,f,k,Aeq,beq,lb,ub) (como no código do professor), usando apenas cvxopt:
#(https://github.com/nolfwin/cvxopt_quadprog/blob/master/cvxopt_qp.py)

def quadprog(H, f, L=None, k=None, Aeq=None, beq=None, lb=None, ub=None):
    """
    Input: Numpy arrays, the format follows MATLAB quadprog function: https://www.mathworks.com/help/optim/ug/quadprog.html
    Output: Numpy array of the solution
    """
    n_var = H.shape[1]

    P = cvxopt.matrix(H, tc='d')
    q = cvxopt.matrix(f, tc='d')

    if L is not None or k is not None:
        assert(k is not None and L is not None)
        if lb is not None:
            L = np.vstack([L, -np.eye(n_var)])
            k = np.vstack([k, -lb])

        if ub is not None:
            L = np.vstack([L, np.eye(n_var)])
            k = np.vstack([k, ub])

        L = cvxopt.matrix(L, tc='d')
        k = cvxopt.matrix(k, tc='d')

    if Aeq is not None or beq is not None:
        assert(Aeq is not None and beq is not None)
        Aeq = cvxopt.matrix(Aeq, tc='d')
        beq = cvxopt.matrix(beq, tc='d')

    sol = cvxopt.solvers.qp(P, q, L, k, Aeq, beq)

    return np.array(sol['x'])

In [175]:
K

array([[192.17939916, 156.60204485, 188.15397044, ..., 175.74449292,
        166.37317168, 180.08389779],
       [156.60204485, 162.28793752, 163.93736749, ..., 158.37845989,
        157.08481758, 174.26891166],
       [188.15397044, 163.93736749, 206.67247758, ..., 177.14358321,
        170.78608457, 179.46321814],
       ...,
       [175.74449292, 158.37845989, 177.14358321, ..., 178.25180459,
        165.59847257, 176.58195473],
       [166.37317168, 157.08481758, 170.78608457, ..., 165.59847257,
        168.64689618, 171.30667885],
       [180.08389779, 174.26891166, 179.46321814, ..., 176.58195473,
        171.30667885, 221.53096267]])

In [177]:
#H = K ; f = c ; L = A ; k = b ; Aeq = Aeq ; beq = beq ; lb = liminf ; ub = limsup
# A e b não tem, devem entrar como vazios
quadprog(K, c, [], [], Aeq, beq, liminf, limsup)

# quadprog(K, c, Aeq, beq, lb=liminf, ub=limsup)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 100